In [1]:
from datetime import datetime,timedelta
from urllib.parse import urlencode
import pandas as pd
import requests
import re
import time
base_url = 'https://datainterface.eastmoney.com/EM_DataCenter/JS.aspx?'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}

query = {'type': 'NS',
         'sty' : 'NSFR',
         'st' : '1',
         'sr' : '-1',
         'p' : '1',
         'ps' : '5000',
         'js' : 'var IBhynDx={pages:(pc),data:[(x)]}',
         'mkt' : '1',
         'rt' : '53721774'
        }

def date_gen():
    
    query = {'type': 'NS',
         'sty' : 'NSFR',
         'st' : '1',
         'sr' : '-1',
         'p' : '1',
         'ps' : '5000',
         'js' : 'var IBhynDx={pages:(pc),data:[(x)]}',
         'mkt' : '1',
         'rt' : '53721774'
        }
    start = datetime.strptime('2017-01-05','%Y-%m-%d').date()
    while start < datetime.today().date():
        query['fd'] = start
        url = base_url + urlencode(query)
        yield url
        start += timedelta(days=7)


main_data = []
for url in date_gen():
    print('Downloading date:',url)
    rs = requests.get(url,headers=headers)
    if rs.text == '':
        continue
    js = rs.text.split('var IBhynDx={pages:1,data:')[1]
    data = eval(js[:-1])
    main_data.extend(data)
    time.sleep(2)

In [65]:
temp = [i.split(',') for i in main_data]
columns = ['会计师事务所','保荐代表人','保荐机构','xxx','律师事务所','日期','所属行业','板块','是否提交财务自查报告',
'注册地','类型','机构名称','签字会计师','签字律师','时间戳','简称']
df = pd.DataFrame(temp,columns=columns)
df['文件链接'] = df['时间戳'].apply(lambda x: "https://notice.eastmoney.com/pdffile/web/H2_" + x + "_1.pdf")
df = df[['机构名称', '类型', '板块', '注册地', '保荐机构','保荐代表人', '律师事务所', '签字律师','会计师事务所', 
'签字会计师', '是否提交财务自查报告', '所属行业','日期','xxx', '时间戳', '保荐机构','文件链接']]
df = df[df['板块'] != '创业板']
# df.to_csv('C:/Users/chen/Desktop/IPO_info/eastmoney_pre_data.csv',index=False,encoding='utf-8-sig')

In [68]:
df['是否提交财务自查报告']=df['是否提交财务自查报告'].replace('','是')
df.drop_duplicates(subset = ['机构名称','是否提交财务自查报告','类型'], keep = 'first', inplace = True) 
df.to_csv('C:/Users/chen/Desktop/IPO_info/eastmoney_pre_data_new.csv',index=False,encoding='utf-8-sig')
# df.loc[df['机构名称'] == '爱柯迪']

In [58]:
df.loc[df['机构名称'] == '爱柯迪']

,机构名称,类型,板块,注册地,保荐机构,保荐代表人,律师事务所,签字律师,会计师事务所,签字会计师,是否提交财务自查报告,所属行业,日期,xxx,时间戳,保荐机构,文件链接
13,爱柯迪,已受理,主板企业,宁波,国金证券股份有限公司,吴小鸣&sbquo;谭军,通力律师事务所,翁晓健&sbquo;李仲英,瑞华会计师事务所(特殊普通合伙),杨峰安&sbquo;任明果,否,汽车制造业,2017/1/5,80452535,AN201711051015446277,国金证券股份有限公司,https://notice.eastmoney.com/pdffile/web/H2_AN...
10063,爱柯迪,已受理,主板企业,宁波,国金证券股份有限公司,吴小鸣&sbquo;谭军,通力律师事务所,翁晓健&sbquo;李仲英,瑞华会计师事务所(特殊普通合伙),杨峰安&sbquo;任明果,是,汽车制造业,2017/5/4,80452535,AN201711051015446277,国金证券股份有限公司,https://notice.eastmoney.com/pdffile/web/H2_AN...
12957,爱柯迪,已反馈,主板企业,宁波,国金证券股份有限公司,吴小鸣&sbquo;谭军,通力律师事务所,翁晓健&sbquo;李仲英,瑞华会计师事务所(特殊普通合伙),杨峰安&sbquo;任明果,是,汽车制造业,2017/6/8,80452535,AN201711051015446277,国金证券股份有限公司,https://notice.eastmoney.com/pdffile/web/H2_AN...
22177,爱柯迪,预先披露更新,主板企业,宁波,国金证券股份有限公司,吴小鸣&sbquo;谭军,通力律师事务所,翁晓健&sbquo;李仲英,瑞华会计师事务所(特殊普通合伙),杨峰安&sbquo;任明果,是,汽车制造业,2017/9/21,80452535,AN201711051015446277,国金证券股份有限公司,https://notice.eastmoney.com/pdffile/web/H2_AN...
22752,爱柯迪,已通过发审会,主板企业,宁波,国金证券股份有限公司,吴小鸣&sbquo;谭军,通力律师事务所,翁晓健&sbquo;李仲英,瑞华会计师事务所(特殊普通合伙),杨峰安&sbquo;任明果,是,汽车制造业,2017/9/28,80452535,AN201711051015446277,国金证券股份有限公司,https://notice.eastmoney.com/pdffile/web/H2_AN...
23913,爱柯迪,已通过发审会,主板企业,宁波,国金证券股份有限公司,吴小鸣&sbquo;谭军,通力律师事务所,翁晓健&sbquo;李仲英,瑞华会计师事务所(特殊普通合伙),杨峰安&sbquo;任明果,,汽车制造业,2017/10/19,80452535,AN201711051015446277,国金证券股份有限公司,https://notice.eastmoney.com/pdffile/web/H2_AN...


In [180]:
from urllib.parse import urljoin
urljoin('http://www.example.com/hithere/something/els.html','./index.html')

'http://www.example.com/hithere/something/index.html'

In [70]:
rssss = requests.get('http://www.csrc.gov.cn/pub/newsite/fxjgb/scgkfxfkyj/index_2.html',headers=headers)


In [72]:
rssss.encoding='utf-8'

In [116]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(rxt.text,'html.parser')

[<a href="./202101/t20210108_390461.html" target="_blank" title="北京菜市口百货股份有限公司首次公开发行股票申请文件反馈意见">北京菜市口百货股份有限公司首次公开发行股票申请文件反馈意见</a>,
 <a href="./202101/t20210108_390460.html" target="_blank" title="大通（福建）新材料股份有限公司首次公开发行股票并上市申请文件反馈意见">大通（福建）新材料股份有限公司首次公开发行股票并上市申请文件反馈意见</a>,
 <a href="./202101/t20210108_390459.html" target="_blank" title="北京金房暖通节能技术股份有限公司首次公开发行股票申请文件反馈意见">北京金房暖通节能技术股份有限公司首次公开发行股票申请文件反馈意见</a>,
 <a href="./202101/t20210108_390458.html" target="_blank" title="青岛食品股份有限公司首次公开发行股票申请文件反馈意见">青岛食品股份有限公司首次公开发行股票申请文件反馈意见</a>,
 <a href="./202012/t20201231_389997.html" target="_blank" title="美庐生物科技股份有限公司首次公开发行股票申请文件反馈意见">美庐生物科技股份有限公司首次公开发行股票申请文件反馈意见</a>,
 <a href="./202012/t20201231_389996.html" target="_blank" title="湖南华联瓷业股份有限公司首次公开发行股票申请文件反馈意见">湖南华联瓷业股份有限公司首次公开发行股票申请文件反馈意见</a>,
 <a href="./202012/t20201225_389452.html" target="_blank" title="浙江盛泰服装集团股份有限公司首次公开发行股票申请文件反馈意见">浙江盛泰服装集团股份有限公司首次公开发行股票申请文件反馈意见</a>,
 <a href="./202012/t20201225_389451.html" target="_blank" title="双枪科技股份有限公司

'25'

In [146]:
url = 'http://www.csrc.gov.cn/pub/newsite/fxjgb/scgkfxfkyj/' + './202101/t20210108_390461.html'

In [147]:
rxt = requests.get(url)

In [148]:
rxt.encoding='utf-8'

In [172]:
re.findall(r'file_appendix=\'<a href=\"(.*)\">',rxt.text)[0].split('.')

['', '/P020210108650707495961', 'docx']

In [165]:
sss.findAll(attrs={'class':'time'})

[<div class="time">
 <span>中国证监会 www.csrc.gov.cn</span>
 <span>时间：2021-01-08</span>
 <span>来源：</span>
 </div>]

In [200]:
from docx import Document
baseUrl = 'http://www.csrc.gov.cn/pub/newsite/fxjgb/scgkfxfkyj/'

def download_page(nexturl):
    r = requests.get(nexturl,headers= headers)
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text,'html.parser')
    projList = soup.find(id='myul').findAll('a')
    for proj in projList:
        href= proj['href']
        docUrl = baseUrl + href
        title = proj.text
        print('fetching: ',title)

        pageInfo = requests.get(docUrl,headers=headers)
        pageInfo.encoding='utf-8'
        docLink = re.findall(r'file_appendix=\'<a href=\"(.*)\">',pageInfo.text)[0]
        doc = requests.get(urljoin(docUrl,docLink),headers=headers,stream=True)
        with open('C:/Users/chen/Desktop/IPO_info/data/证监会文件/{}.docx'.format(title),'wb') as f:
                f.write(doc.content)
        time.sleep(2)

r = requests.get(baseUrl,headers= headers)
r.encoding = 'utf-8'
soup = BeautifulSoup(r.text,'html.parser')
numPage = re.findall(r'var countPage = (.*)//',soup.text)[0]
numPage = int(numPage)
download_page(baseUrl)

for i in range(1,1):
    nextUrl = baseUrl + 'index_{}.html'.format(i)
    download_page(nextUrl)

fetching:  北京菜市口百货股份有限公司首次公开发行股票申请文件反馈意见
fetching:  大通（福建）新材料股份有限公司首次公开发行股票并上市申请文件反馈意见
fetching:  北京金房暖通节能技术股份有限公司首次公开发行股票申请文件反馈意见
fetching:  青岛食品股份有限公司首次公开发行股票申请文件反馈意见
fetching:  美庐生物科技股份有限公司首次公开发行股票申请文件反馈意见
fetching:  湖南华联瓷业股份有限公司首次公开发行股票申请文件反馈意见
fetching:  浙江盛泰服装集团股份有限公司首次公开发行股票申请文件反馈意见
fetching:  双枪科技股份有限公司首次公开发行股票申请文件反馈意见
fetching:  广东洪兴实业股份有限公司首次公开发行股票申请文件反馈意见
fetching:  广东中旗新材料股份有限公司首次公开发行股票申请文件反馈意见
fetching:  绿田机械股份有限公司首次公开发行股票申请文件反馈意见
fetching:  康定金鑫矿业旅游股份有限公司首次公开发行股票申请文件反馈意见
fetching:  上海十方生态园林股份有限公司首次公开发行股票申请文件反馈意见
fetching:  永道射频技术股份有限公司首次公开发行股票申请文件反馈意见
fetching:  中铁特货物流股份有限公司首次公开发行股票申请文件反馈意见
fetching:  上海新农科技股份有限公司首次公开发行股票申请文件反馈意见
fetching:  重庆三峡银行股份有限公司首次公开发行股票申请文件反馈意见
fetching:  普洱澜沧古茶股份有限公司首次公开发行股票申请文件反馈意见
fetching:  扬州日兴生物科技股份有限公司首次公开发行股票并上市申请文件反馈意见
fetching:  南京冠石科技股份有限公司首次公开发行股票申请文件反馈意见


In [199]:
docUrl ='http://www.csrc.gov.cn/pub/newsite/fxjgb/scgkfxfkyj/202011/t20201120_386566.html'
PageInfo = requests.get(docUrl,headers=headers)
pageInfo.encoding='utf-8'
docLink = re.findall(r'file_appendix=\'<a href=\"(.*)\">',PageInfo.text)[0]
print(docLink)
doc = requests.get(urljoin(docUrl,docLink),headers=headers)
with open('C:/Users/chen/Desktop/IPO_info/data/证监会文件/{}.doc'.format(title),'wb') as f:
        f.write(doc.content)
time.sleep(2)

./P020201120637455060176.doc
